In [1]:
import json

import tensorflow as tf

import numpy as np
import random
import pickle 
import matplotlib.pyplot as plt
from collections import Counter

 
import vist
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)


In [2]:
with open("data/caltech_stories_dataset", 'rb') as file:
        caltech_stories_dataset = pickle.load(file)
        

type(caltech_stories_dataset)
labels = [1] * 12000
labels[7200:] = [0] * 4800

caltech_stories_dataset = np.array(caltech_stories_dataset)
labels = np.array(labels)

#shuffle and create datasets
indices = list(range(12000))
random.shuffle(indices)

labels = labels[indices]
caltech_stories_dataset = caltech_stories_dataset[indices]


train = {"features" : {"filenames" : caltech_stories_dataset[:10000,:]}, "labels" : labels[:10000]}
valid = {"features" : {"filenames" : caltech_stories_dataset[10000:11000,:]}, "labels" : labels[10000:11000]}
test = {"features" : {"filenames" : caltech_stories_dataset[11000:,:]}, "labels" : labels[11000:]}

caltech_data = {"training": train, "validation": valid, "test" : test}
print("Labeled and segmented the dataset into training/validation/test sets")

Labeled and segmented the dataset into training/validation/test sets


In [3]:
with open("data/caltech_features", 'rb') as file:
        caltech_features = pickle.load(file)
        
        
for _set in caltech_data:
    features = caltech_data[_set]["features"]
    filenames = features["filenames"]
    features["fc7_features"] = []
    features["fc8_features"] = []
    
    for story in filenames:
        
        fc7_features_story = []
        fc8_features_story = []
        
        for img_filename in story:
            tokens = img_filename.split("/")
            class_name = tokens[-2]
            img_name = tokens[-1] 
            
            fc7 = caltech_features[class_name][img_name]["fc7"]
            fc8 = caltech_features[class_name][img_name]["fc8"]
            
            fc7_features_story.append(fc7) 
            fc8_features_story.append(fc8)
        
        features["fc7_features"].append(fc7_features_story) 
        features["fc8_features"].append(fc8_features_story)

print("Loaded fc7 and fc8 features")

Loaded fc7 and fc8 features


In [4]:
print(caltech_data.keys())
print(caltech_data['training'].keys())
print(caltech_data['training']['features'].keys())


dict_keys(['test', 'training', 'validation'])
dict_keys(['labels', 'features'])
dict_keys(['fc7_features', 'fc8_features', 'filenames'])


In [53]:
import itertools  

train_features = caltech_data['training']['features']['fc8_features']
train_features = [list(itertools.chain(*story_features)) for story_features in train_features]
train_features = np.array(train_features)
bin_label = caltech_data['training']['labels']
train_labels = np.zeros([10000,2]) 
train_labels[np.arange(10000), np.array(bin_label).reshape(1,10000)] = 1

validation_features = caltech_data['validation']['features']['fc8_features']
validation_features = [list(itertools.chain(*story_features)) for story_features in validation_features]
validation_features = np.array(validation_features)
bin_label = caltech_data['validation']['labels']
validation_labels = np.zeros([1000,2]) 
validation_labels[np.arange(1000),np.array(bin_label).reshape(1,1000)] = 1


test_features = caltech_data['test']['features']['fc8_features']
test_features = [list(itertools.chain(*story_features)) for story_features in test_features]
test_features = np.array(test_features)
bin_label = caltech_data['test']['labels']
test_labels = np.zeros([1000,2]) 
test_labels[np.arange(1000), np.array(bin_label).reshape(1,1000)] = 1




In [58]:


def get_batch(X, Y, current_batch, batch_size):
    batch_x = X[current_batch*batch_size : (current_batch+1)*batch_size, :]
    batch_y = Y[current_batch*batch_size : (current_batch+1)*batch_size, :] 
    return batch_x, batch_y

# Parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 100
display_step = 1

# Network Parameters
n_input = 5000 # MNIST data input (img shape: 28*28)
n_hidden_1 = 512 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

keep_prob = tf.placeholder(tf.float32)


# Create model
def multilayer_perceptron(x, weights, biases, keep_prob):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
 


# Construct model
pred = multilayer_perceptron(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train_features.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = get_batch(train_features, train_labels, i, batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y,
                                                          keep_prob: 0.5})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print(" Train accuracy:", accuracy.eval(feed_dict = {x: train_features, y: train_labels, keep_prob : 1.0}))
            
            print(" Validation accuracy:", accuracy.eval(feed_dict = {x: validation_features, y: validation_labels, keep_prob : 1.0}))
    print("Optimization Finished!")

    # Test model
    print("Accuracy:", accuracy.eval(feed_dict ={x: test_features, y: test_labels, keep_prob : 1.0}))


Epoch: 0001 cost= 2552.493851871
 Train accuracy: 0.9699
 Validation accuracy: 0.971
Epoch: 0002 cost= 522.033785839
 Train accuracy: 0.9871
 Validation accuracy: 0.99
Epoch: 0003 cost= 260.157989750
 Train accuracy: 0.9956
 Validation accuracy: 0.996
Epoch: 0004 cost= 202.556279044
 Train accuracy: 0.9967
 Validation accuracy: 0.998
Epoch: 0005 cost= 123.793089790
 Train accuracy: 0.9972
 Validation accuracy: 0.997
Epoch: 0006 cost= 126.215665274
 Train accuracy: 0.9977
 Validation accuracy: 0.998
Epoch: 0007 cost= 79.172372742
 Train accuracy: 0.9988
 Validation accuracy: 0.998
Epoch: 0008 cost= 64.526698022
 Train accuracy: 0.9976
 Validation accuracy: 0.996
Epoch: 0009 cost= 58.054938221
 Train accuracy: 0.9995
 Validation accuracy: 0.999
Epoch: 0010 cost= 49.708076024
 Train accuracy: 0.9986
 Validation accuracy: 0.998
Optimization Finished!
Accuracy: 0.997
